<a href="https://colab.research.google.com/github/TakuroTerui/PyOptBook/blob/main/%E6%95%B0%E7%90%86%E6%9C%80%E9%81%A9%E5%8C%96_ch6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ohmsha/PyOptBook

Cloning into 'PyOptBook'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 206 (delta 82), reused 71 (delta 46), pack-reused 76
Receiving objects: 100% (206/206), 2.31 MiB | 8.22 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [3]:
cd PyOptBook

/content/PyOptBook


In [5]:
import pandas as pd

students_df = pd.read_csv('6.api/resource/students.csv')
print(students_df.shape)

(24, 4)


In [6]:
# 上位5件を表示
students_df.head()

,student_id,license,gender,grade
0,0,0,0,1
1,1,1,1,2
2,2,1,0,3
3,3,1,1,4
4,4,0,1,1


In [7]:
cars_df = pd.read_csv('6.api/resource/cars.csv')
print(cars_df.shape)

(6, 2)


In [8]:
cars_df

,car_id,capacity
0,0,6
1,1,6
2,2,5
3,3,4
4,4,5
5,5,5


In [9]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 64.2 MB/s eta 0:00:00


### 数理モデルの実装

In [11]:
import pandas as pd
import pulp

students_df = pd.read_csv('6.api/resource/students.csv')
cars_df = pd.read_csv('6.api/resource/cars.csv')

# 学生の乗車グループ分け問題（0-1整数計画問題）のインスタンス作成
prob = pulp.LpProblem('ClubCarProblem', pulp.LpMinimize)

# リスト
# 学生のリスト
S = students_df['student_id'].to_list()
# 車のリスト
C = cars_df['car_id'].to_list()
# 学年のリスト
G = [1, 2, 3, 4]
# 学生と車のペアのリスト
SC = [(s, c) for s in S for c in C]
# 免許を持っている学生のリスト
S_license = students_df[students_df['license'] == 1]['student_id']
# 学年がgの学生のリスト
S_g = {g:students_df[students_df['grade'] == g]['student_id'] for g in G}
# 男性と女性のリスト
S_male = students_df[students_df['gender'] == 0]['student_id']
S_female = students_df[students_df['gender'] == 1]['student_id']

# 定数
# 車の乗車定員の定数
U = cars_df['capacity'].to_list()

# 変数
# 学生をどの車に割り当てるかを変数として定義
x = pulp.LpVariable.dicts('x', SC, cat='Binary')

# 制約
# (1) 各学生を1つの車に割り当てる
for s in S:
    prob += pulp.lpSum([x[s, c] for c in C]) == 1

# (2) 法規制に関する制約：各車に乗車定員より多く乗ることができない
for c in C:
    prob += pulp.lpSum([x[s, c] for s in S]) <= U[c]

# (3) 法規制に関する制約：各車にドライバーを一人以上割り当てる
for c in C:
    prob += pulp.lpSum([x[s, c] for s in S_license]) >= 1

# (4) 懇親を目的とした制約：各車に各学年の学生を一人以上割り当てる
for c in C:
    for g in G:
        prob += pulp.lpSum([x[s, c] for s in S_g[g]]) >= 1
    
# (5) ジェンダーバランスを考慮した制約：各車に男性を一人以上割り当てる
for c in C:
    prob += pulp.lpSum([x[s, c] for s in S_male]) >= 1

# (6) ジェンダーバランスを考慮した制約：各車に女性を一人以上割り当てる
for c in C:
    prob += pulp.lpSum([x[s, c] for s in S_female]) >= 1

### 問題を解く

In [12]:
# 求解
status = prob.solve()
print('Status:', pulp.LpStatus[status])

Status: Optimal


In [14]:
# 最適化結果の表示
# 各車に割り当てられている学生のリストを辞書に格納(車ID→割り当てられた学生のリスト
car2students = {c: [s for s in S if x[s, c].value() == 1] for c in C}

# 各車の乗車定員（車ID→乗車定員）
max_people = dict(zip(cars_df['car_id'], cars_df['capacity']))

for c, ss in car2students.items():
    print(f'車ID：{c}')
    print(f'学生数(乗車定員)：{len(ss)}({max_people[c]})')
    print(f'学生：{ss}¥n')

車ID：0
学生数(乗車定員)：4(6)
学生：[6, 7, 8, 9]¥n
車ID：1
学生数(乗車定員)：4(6)
学生：[4, 18, 21, 23]¥n
車ID：2
学生数(乗車定員)：4(5)
学生：[5, 19, 20, 22]¥n
車ID：3
学生数(乗車定員)：4(4)
学生：[3, 10, 13, 16]¥n
車ID：4
学生数(乗車定員)：4(5)
学生：[0, 2, 11, 17]¥n
車ID：5
学生数(乗車定員)：4(5)
学生：[1, 12, 14, 15]¥n
